As of 1/5/2024: DOWNLOAD THE DATA IN UTC TIME! Otherwise daylight savings will break the script

This is a working draft that is making improvements to the "VOC_Active_2023_1024" notebook.

That one works fine, but it can be improved with some script modification and additional markup cells with comments/instructions.

Major improvements needed include: 
* limit data to complete 24-hour periods, beginning and ending at midnight (cut off partial start and end days);
* create a df of 1-min averages towards beginning of script to cut down on unneccesary processing time; 
* issues with processing data due to daylight savings (if study period spans the time shift, the code breaks unless the raw data is in UTC); 
* fix resampling (as long as we have at least one reading every minute, that is satisfactory for the summaries, graphs, and completeness assessment - the 30-sec resample may be an unnecessary step)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Set the location to where you have saved the CSV file.
location = 'VOC_snippet.csv'

# Set the time zone first. Here are some examples: America/New_York, America/Los_Angeles, America/Chicago, Pacific/Honolulu.
# The format generally follows the pattern "America/Region" or "Continent/City".
timezone = 'America/New_York' 

# Read the CSV file and create a DataFrame
df = pd.read_csv(location)
# df = pd.read_csv(location, parse_dates=["Iso8601Time"])

# Print a preview of the beginning and end of df in order to determine 
# which data is in each column and the start and end times of recorded data
print(df.head(3))
# print(df.tail())
df.info()

# Replace with the actual original filename without extension
# Make this more flexible code. User should not have to enter this. Do a string edit of the location variable
original_filename = 'QQQQQQ' 

                        Iso8601Time  3579.feed_161522.tvoc_ch0
0  2023-10-25T04:57:47.081000-04:00                         65
1  2023-10-25T04:57:48.081000-04:00                         48
2  2023-10-25T04:57:49.080000-04:00                         57
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097796 entries, 0 to 1097795
Data columns (total 2 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   Iso8601Time                1097796 non-null  object
 1   3579.feed_161522.tvoc_ch0  1097796 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 16.8+ MB


In [2]:
print(pd.__version__)

2.2.1


Yvonne comments on cell below:

Can you get rid of any unnecessary commented code once this is finished? It looks like the first batch of code that is all hashmarked is code from the first draft that you commented out. 

Does the following code trim the data so that we use the data starting at 12am on the first day and ending at 11:59 pm (or 23:59) on the last day for our analysis?

It looks like right now the code is printing the "Percentage of missing data...." message for every hour rather than for just the hours that have more than 25% of their data missing. 

In [174]:
# Convert the values in the first column to datetime format

# df.info()
# print(df['Iso8601Time'][0])
# print(df.head(3))

df.iloc[0:922924, 0] = pd.to_datetime(df.iloc[0:922924, 0], format='%Y-%m-%dT%H:%M:%S.%f%z')
# df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format='%Y-%m-%dT%H:%M:%S.%f')
# df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format='%Y-%m-%dT%H:%M:%S.%f%z')
# df.iloc[:, 0] = pd.to_datetime(df["Iso8601Time"], format='%Y-%m-%dT%H:%M:%S.%f%z')
# df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format='ISO8601', utc=True)

print('=========')
# print(df['Iso8601Time'][0])
print(df.head(3))
print('---------')

                        Iso8601Time  3579.feed_161522.tvoc_ch0
0  2023-10-25 04:57:47.081000-04:00                         65
1  2023-10-25 04:57:48.081000-04:00                         48
2  2023-10-25 04:57:49.080000-04:00                         57
---------


In [175]:
df.info()
print('++++++++')
print(df['Iso8601Time'][0])
print(df['Iso8601Time'][0].tz_convert('America/New_York'))
# df.set_index(df.columns[0], inplace=True)
# df.head(3)

# df['Iso8601Time'].tz_convert('America/New_York')

print('++++++++')
print(df.head(3))

# df['Iso8601Time'][0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097796 entries, 0 to 1097795
Data columns (total 2 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   Iso8601Time                1097796 non-null  object
 1   3579.feed_161522.tvoc_ch0  1097796 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 16.8+ MB
++++++++
2023-10-25 04:57:47.081000-04:00
2023-10-25 04:57:47.081000-04:00
++++++++
                        Iso8601Time  3579.feed_161522.tvoc_ch0
0  2023-10-25 04:57:47.081000-04:00                         65
1  2023-10-25 04:57:48.081000-04:00                         48
2  2023-10-25 04:57:49.080000-04:00                         57


In [161]:
# Convert the values in the first column to datetime format
# df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format='%Y-%m-%dT%H:%M:%S.%f')
df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0], format='%Y-%m-%dT%H:%M:%S.%f%z')

df.info()

# Set the first column as the index
df.set_index(df.columns[0], inplace=True)

# Set the time zone
df = df.tz_convert(timezone)

# Find the first and last valid timestamps in the DataFrame
first_timestamp = df.index.min()
last_timestamp = df.index.max()

# Round down the first timestamp to the nearest hour and add 1 hour to it to set it as the desired start time
desired_start_time = first_timestamp.floor('H') + pd.Timedelta(hours=1)

# Round down the last timestamp to the nearest hour to set it as the desired end time
desired_end_time = last_timestamp.floor('H')

# Filter the DataFrame to keep rows within the desired start and end times
df = df[(df.index >= desired_start_time) & (df.index <= desired_end_time)]

# reprint head and tail to verify correct start and end times
print(df.head())
print(df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097796 entries, 0 to 1097795
Data columns (total 2 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   Iso8601Time                1097796 non-null  object
 1   3579.feed_161522.tvoc_ch0  1097796 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 16.8+ MB


TypeError: index is not a valid DatetimeIndex or PeriodIndex

In [35]:
# Filter the DataFrame to remove negative values
df = df[df >= 0]

# Calculate the total number of seconds in an hour
total_seconds = 60 * 60

# Loop over each hour of data
# NOTE instead of creating a df over and over again for each hour of data, wouldn't it be more efficient to use 
# something like a count function?
for hour_start in pd.date_range(start=df.index.min(), end=df.index.max(), freq='1H'):
    # Get the hour end time
    hour_end = hour_start + pd.Timedelta(hours=1)
    
    # Subset the data for the current hour
    hour_data = df.loc[(df.index >= hour_start) & (df.index < hour_end)]

    # Resample the data to 1-second intervals and count the number of data points per second
    resampled_df = hour_data.resample('1S').count()

    # Limit the resampled DataFrame to 1 hour of data
    resampled_df = resampled_df.iloc[:total_seconds]

    # Calculate the number of missing seconds
    missing_seconds = total_seconds - resampled_df.iloc[:, 0].sum()

    # Calculate the percentage of missing data
    missing_percentage = (missing_seconds / total_seconds) * 100

    # Print the percentage of missing data for the hour if larger than 25%
    if missing_percentage >= 25:
        print(f"Percentage of missing data in hour from {hour_start} to {hour_end}: {missing_percentage:.2f}%")

# Resample the data to 30-second intervals and calculate the mean
resampled_data_30s = df.resample('30S').mean()

# Count the number of readings equal to or greater than 500 ppb
count = 0
for data in resampled_data_30s.iloc[:, 0]:
    if data >= 500:
        count += 1

# Calculate the percentage
percentage = (count / len(resampled_data_30s.iloc[:, 0])) * 100

# Print the results
print("Number of readings equal to or greater than 500 ppb:", count)
print("Percentage of data exceeding 500 ppb:", format(percentage, '.2f'), "%")
# Would also be helpful to have the dates when those exceedances happened

TypeError: Start and end cannot both be tz-aware with different timezones

Figure 1

Yvonne comments on cell below:

Should we be concerned about the Userwarning regarding the timezone being dropped?


In [ ]:
# It would be better if the analysis and graph were calculated from month-long data
# i.e. July 8 - AUgust 8. Not just July, August, Sept, etc.
# We don't install monitors on the first of the month so July data could just be 5 days!
# X-axis labels would then be "Month1, Month2, etc." or something similar. Also, what is going on with
# the daily averages here? There should be no partial days used. Is that happening?

# Calculate the daily averages
daily_averages = resampled_data_30s.resample('D').mean()

# Resample the daily averages to monthly intervals and calculate the mean
monthly_averages = daily_averages.resample('M').mean()

# Convert the index to a PeriodIndex with timezone awareness
daily_averages.index = daily_averages.index.to_period('D')

# Calculate the standard deviation of daily values from the monthly average
std_values = daily_averages.groupby(daily_averages.index.to_timestamp('M')).std()

# Find the month with the highest average
highest_month = monthly_averages.iloc[:, 0].idxmax()
highest_average = monthly_averages.iloc[:, 0].max()
std_dev = monthly_averages.iloc[:, 0].std()

# Print the result
print(f"Highest monthly average was in {highest_month.strftime('%B %Y')} at {highest_average:.3f} ± {std_dev:.3f} ppb")

# Set the figure DPI
plt.rcParams['figure.dpi'] = 100

# Generate the error bar plot
plt.errorbar(monthly_averages.index, monthly_averages.iloc[:, 0], yerr=std_values.iloc[:, 0], fmt='o', capsize=15, color='black')

# Format the x-axis labels
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator())

# Adjust the location of x-axis labels
plt.gca().set_xticks(monthly_averages.index)  # Set the tick positions to match the x-values

# Adjust the x-axis labels rotation
plt.xticks(rotation=45, fontsize=7)

# Adjust the y-axis tick spacing
plt.yticks(range(0, 300, 50), fontsize=7)

# Set the font for axis labels
plt.gca().set_xlabel('Month', fontname='Times New Roman', fontsize=9)
plt.gca().set_ylabel('VOC Concentration (ppb)', fontname='Times New Roman', fontsize=9)

# Set the font for title
plt.gca().set_title('Monthly Average VOC Levels', fontname='Times New Roman', fontsize=10)

# Save the figure
plt.savefig('VOC Month Avg.png', dpi=300)

# Adjust the figure size (modify width and height as needed)
plt.figure(figsize=(10, 6))

# Show the plot
plt.show()

In [ ]:
monthly_mean = float(monthly_averages.mean())
print("the mean of monthly averages is {}".format(monthly_mean))

Figure 2

Yvonne notes on cell below:

In the third block you have comments "Notice: please specify the start(end) date here"  Aren't these variables/dates defined in the first block/cell above?

For the y axis ticks, as written in the script, will we have to modify the line below defining y-axis tick spacing if our data has a different range? FOr example, if our VOC levels are above 600? If so, is there a way to automatically set this (without the user having to define it) based on the maximum value of these daily averages, so that it would be the nearest multiple of 100 above the daily average value that is being plotted?



In [ ]:
import matplotlib.pyplot as plt

# FOR EACH FILE, Specify the date period here for the Daily Avg graph
# This should omit the partial start and end dates
start_date = '2023-09-27'
end_date = '2024-01-21'
start_date = pd.Timestamp(start_date, tz=timezone) 
end_date = pd.Timestamp(end_date, tz=timezone)

# Resample the data to daily intervals and calculate the mean
grouped = resampled_data_30s.resample('D').mean()
std_values = resampled_data_30s.resample('D').std()

# Select the desired date range
selected_grouped = grouped[start_date:end_date]
selected_std_values = std_values[start_date:end_date]

# Configure plot settings
plt.rcParams['figure.dpi'] = 100
plt.figure(figsize=(10, 6))

# Generate the line plot with connected data points
plt.plot(selected_grouped.index, selected_grouped.iloc[:, 0], marker='o', linestyle='-', color='black', linewidth=0.3)

# Add error bars
plt.errorbar(selected_grouped.index, selected_grouped.iloc[:, 0], yerr=selected_std_values.iloc[:, 0], fmt='none', color='black', capsize=15)

# create the x-labels variable to use for x-tick formatting
x_labels = [date.strftime('%m-%d') for date in selected_grouped.index]

# Format the x-axis labels
plt.xticks(selected_grouped.index[::5], x_labels[::5] , rotation=45, fontsize=7)

## Format the x-axis labels (original script)
#plt.xticks(selected_grouped.index, [date.strftime('%m-%d') for date in selected_grouped.index], rotation=45, fontsize=7)

# Set the x-axis tick positions to match the x-values
plt.gca().set_xticks(selected_grouped.index)

# Calculate the maximum value of daily averages
max_daily_average = selected_grouped.iloc[:, 0].max()

# Default is to set the y-axis tick spacing at 100, we often need to increase this to accomodate data
y_tick_spacing = 100

# Calculate the y-axis tick range dynamically
# NOTE we need to change this so the range includes the maximum 30s value
#y_tick_range = range(0, int(max_daily_average) + y_tick_spacing, y_tick_spacing)

y_tick_range = range(0, 800 + y_tick_spacing, y_tick_spacing)

# Adjust the y-axis tick spacing and font size
plt.yticks(y_tick_range, fontsize=7)

# Set the x-axis and y-axis labels
plt.xlabel('Date', fontname='Times New Roman', fontsize=9)
plt.ylabel('VOC Concentration (ppb)', fontname='Times New Roman', fontsize=9)

# Set the title with the start and end dates
title = f"Daily Average VOC Levels: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}"
plt.title(title, fontname='Times New Roman', fontsize=10)

# Save the plot
plt.savefig('VOC Daily Avg.png', dpi=300)

# Show the plot
plt.show()
print("The highest daily average is {}".format(max_daily_average))

Yvonne comments on cell below:

The calculation and reporting of readings above 500 ppb should be done for the total time period (from the first full day to the last full day of the data collected),  not the selected time period used to generate the graph below. 

In [ ]:
import matplotlib.pyplot as plt

# FOR EACH FILE, specify the date and time period here for the VOC by minute graph
start_datetime = '2023-12-23 03:05'
end_datetime = '2023-12-23 05:55'
start_datetime = pd.Timestamp(start_datetime, tz=timezone)
end_datetime = pd.Timestamp(end_datetime, tz=timezone)


## Right now this is graphing 30s readings. It should be graphing 1 minute readings.
# Filter the dataset based on the desired date and time range
filtered_data = resampled_data_30s[(resampled_data_30s.index >= start_datetime) & (resampled_data_30s.index <= end_datetime)]

# Configure plot settings
plt.rcParams['figure.dpi'] = 100
plt.figure(figsize=(10, 6))

# Create the line plot
plt.plot(filtered_data.index, filtered_data.iloc[:, 0], color='black', linewidth=1)

# Extract the time component from the filtered_data index
x_labels = [date.strftime('%H:%M') for date in filtered_data.index]

# Set the x-axis tick labels
plt.xticks(filtered_data.index[::20], x_labels[::20], rotation=45, fontsize=7)  # Display every 10th label

# Set the y-axis tick labels font size
plt.yticks(fontsize=7)

# Set the x-axis label
plt.xlabel('Time', fontname='Times New Roman', fontsize=9)

# Set the y-axis label
plt.ylabel('VOC Concentration (ppb)', fontname='Times New Roman', fontsize=9)

# Set the title with the specified date
title = f"VOC Levels Per Minute on {start_datetime.strftime('%Y-%m-%d')}"
plt.title(title, fontname='Times New Roman', fontsize=10)

# Save the plot
plt.savefig('VOC by Minute.png', dpi=300)

# Show the plot
plt.show()

# Resample the data to 1-minute intervals and calculate the mean
resampled_data_1min = df.resample('1T').mean()

# # Fill any missing values (NaN) in the DataFrame by forward-filling (ffill) the previous value
# resampled_data_1min.fillna(method='ffill', inplace=True)

# Filter the dataset based on the desired date and time range
filtered_data_1min = resampled_data_1min[(resampled_data_1min.index >= start_datetime) & (resampled_data_1min.index <= end_datetime)]

# Find the highest 1 minute ppb value
highest_1min_ppb = filtered_data_1min.iloc[:, 0].max()

# Print the result
print("The highest 1 minute ppb value from the time period is {}".format(highest_1min_ppb))

In [ ]:
# Filter the dataset based on the desired date range.
studyperiod_data_1min = resampled_data_1min[(resampled_data_1min.index >= start_date) & (resampled_data_1min.index <= end_date)]

# Find the highest 1 minute ppb value
highest_1min_ppb = studyperiod_data_1min.iloc[:, 0].max()

# Print the result
print("The highest 1 minute ppb value from the entire study period is {}".format(highest_1min_ppb))

In [ ]:
# Not sure what this block is doing and why it is HERE
# The script should set the time to local and resample to 1 minutes averages at the beginning, right?
# We need to handle that 1 minute averaging appropriately (null values)

import pandas as pd
from datetime import datetime

# Convert the index to local time (assuming the original data is in UTC)
resampled_data_1min_local = resampled_data_1min.tz_convert(timezone)  # Replace 'Your_Local_Timezone' with your actual local timezone

# Split the local time index into separate date and time columns
resampled_data_1min_local['Date'] = resampled_data_1min_local.index.date
resampled_data_1min_local['Time'] = resampled_data_1min_local.index.time

# Drop the original datetime index
resampled_data_1min_local.reset_index(drop=True, inplace=True)

# Create a new DataFrame with the desired columns
new_df = pd.DataFrame({
    'Date': resampled_data_1min_local['Date'],
    'Time': resampled_data_1min_local['Time'],
    '1-Minute Averages (ppb)': resampled_data_1min_local.iloc[:, 0]
})

output_filename = f"{original_filename}_processed.csv"

# Save the new DataFrame to a CSV file
new_df.to_csv(output_filename, index=False)

print(f"Processed data saved to {output_filename}.")